In [11]:
from sqlalchemy import create_engine
from os import getenv as ge
from dotenv import load_dotenv
import pandas as pd
from snowflake.sqlalchemy import URL
from pprint import pprint
load_dotenv()

snow_engine = create_engine(URL(
    account = ge('snow_account'),
    user = ge('snow_user'),
    password = ge('snow_pass'),
    database = ge('snow_db'),
    warehouse = ge('snow_wh')
))

# Goal: 
### Identify students per course in each domain.

# Query
```sql
select
    year(lt.start_date) as term_year,
    ifnull(lt.name, 'Unknown Term') as term,
    replace(ifnull(h2.name, 'No Parent'),'NoName', 'Institution') as hierarchy_parent_node,
    ifnull(h1.name, 'No Node') as hierarchy_node,
    count(distinct lc.id) as course_count,
    count(distinct lp.id) as student_count,
    student_count/course_count as students_per_course
from cdm_lms.course lc
inner join cdm_lms.institution_hierarchy_course ihc
    on lc.id = ihc.course_id
    and ihc.primary_ind = 1
    and ihc.row_deleted_time is null
left join cdm_lms.institution_hierarchy h1
    on ihc.institution_hierarchy_id = h1.id
left join cdm_lms.institution_hierarchy h2
    on h1.institution_hierarchy_parent_id = h2.id 
left join cdm_lms.term lt
    on lt.id = lc.term_id
left join cdm_lms.person_course pc
    on lc.id = pc.course_id
left join cdm_lms.person lp
    on pc.person_id = lp.id
where pc.course_role = 'S' and
    term_year is not null
group by
    year(lt.start_date),
    h1.name,
    h2.name,
    lt.name
order by
    year(lt.start_date) desc,
    students_per_course desc,
    lt.name,
    h1.name
```

# Results

In [22]:
with open('./queryfiles/Hack-Assess1.sql') as f:
    query = f.read()
example = pd.read_sql(query, con=snow_engine)

In [24]:
example.head(n=15)

,term_year,term,hierarchy_parent_node,hierarchy_node,course_count,student_count,students_per_course
0,2020,Spring 2020,Institution,Akindi,2,920,460.000000
1,2020,Spring 2020,Institution,SUPP,46,3536,76.869565
2,2020,Summer 2020,Institution,SUPP,21,823,39.190476
3,2020,Spring 2020,Institution,Learning Resources,5,79,15.800000
4,2020,Spring 2020,Institution,Social Work,31,466,15.032258
5,2019,Spring 2019,Institution,SUPP,43,3949,91.837209
6,2019,Fall 2019,Institution,SUPP,58,4604,79.379310
7,2019,Summer 2019,Institution,SUPP,8,267,33.375000
8,2019,Summer 2019,Institution,First Year,3,81,27.000000
9,2019,Fall 2019,Institution,Learning Resources,5,134,26.800000


# Outcomes
First year courses, support and social work are consistently high.  Akindi is relatively new, but huge numbers of students per course.  Knowing this helps identify good partner courses for new tools / lti integrations.